In [ ]:
import cv2
import numpy as np
from PIL import Image

pts1 = np.array([[20,20],[130,70],[90,90],[80,130]],dtype=np.int32)
img = Image.open("./1.jpg")
img = np.array(img)
cv2.polylines(img, [pts1], True, (255, 255, 255), 4)


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# 0으로 채워진 이미지(array) 생성
image = np.zeros((150,150,3),np.uint8)
img = Image.open("./1.jpg")
image = np.array(img)

# 다각형의 좌표를 순서대로 입력, 입력 순서대로 선을 그리게 된다
pts1 = np.array([[783.0, 40.0], [1177.0, 51.0], [1174.0, 200.0], [780.0, 189.0]],dtype=np.int32)

# cv2의 polylines로 다각형 그리기
# cv2.polylines(원본그림, 좌표리스트, 마지막점과 첫점 연결 여부, 선색, 선 굵기)
cv2.polylines(image,[pts1],True,(255,255,255),4)
plt.imshow(image)

In [ ]:
# root = "E:/workspace/paddleocr/outputs/det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune"
# root = "E:/workspace/paddleocr/outputs/det___MobileNetV3_large_x0_5___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune"
root = "E:/workspace/paddleocr/outputs/rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune"
config = root+"/trained_model/config.yml"
checkpoint = root+"/trained_model/best_model/model"
pretrained = root+"/trained_model/best_model/model"
save_inferencd = root+"/trained_model/inference_model"

command = f"python code/PaddleOCR/tools/export_model.py -c {config}  -o Global.pretrained_model={pretrained} Global.save_inference_dir={save_inferencd} Global.checkpoints={checkpoint}"

# import os
# os.system(command)
command

In [22]:
from pathlib import Path
import shutil

path = "E:/workspace/paddleocr/labelsets/ai_hub_korean_image_det_16_04_80/infer_label.txt"
with open(path) as f:
    lines = [line.rstrip('\n') for line in f.readlines()]

for i, line in enumerate(lines[:1000]):
    path = Path("./datasets")/line
    target = f"E:/workspace/paddleocr/temp/{i}.jpg"
    shutil.copy(path, target)
    

In [6]:

from database import *
from pathlib import Path

def full_infer(image_dir, det_model_dir, rec_model_dir, save_dir):
    code = 'code/PaddleOCR/tools/infer/predict_system.py'
    det_algorithm="DB"

    # rec_model_dir = "E:/workspace/paddleocr/outputs/rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune/trained_model/inference_model"
    # rec_char_dict_path = "E:/workspace/paddleocr/code/PaddleOCR/ppocr/utils/dict/korean_dict.txt"
    vis_font_path = "C:\Windows\Fonts\malgun.ttf"

    command = f"""python {code} --image_dir={image_dir} \
    --det_model_dir={det_model_dir} \
    --det_algorithm={det_algorithm} \
    --rec_model_dir={rec_model_dir} \
    --rec_image_shape="3, 32, 100" \
    --rec_char_dict_path={rec_char_dict_path} \
    --vis_font_path={vis_font_path} \
    --draw_img_save_dir={save_dir} \
    --drop_score=0.3 \
    --det_db_box_thresh=0.4"""
    
    path = Path(project.PROJECT_ROOT)/"inference/run.sh"
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a") as f:
        f.write(command+"\n")
    
modeldb = ModelDB()
workdb = WorkDB()

# det_model_dir = modeldb.get_config("MobileNetV3_large_x0_5")["inference_model_dir"] 
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/Mobile_PaddleOCR_pretrained"
# full_infer(det_model_dir, rec_model_dir, save_dir)

# image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
# det_model_dir = modeldb.get_config("ml_PP-OCRv3_det")["inference_model_dir"]
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PaddleOCR_PaddleOCR_pretrained_sangmu"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_sangmu"
full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

image_dir = "./datasets/ai_hub_korean_det_partial_test/images"
det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_aihub"
full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)



./works/rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune/inference_model/best


In [42]:
modeldb.get_all_id()

['en_PP-OCRv3_det',
 'korean_PP-OCRv3_rec',
 'ml_PP-OCRv3_det',
 'MobileNetV3_large_x0_5']